# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time
from sqlalchemy import create_engine
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import make_scorer, accuracy_score, recall_score, precision_score, fbeta_score, classification_report
import pickle




%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\trekd\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\trekd\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\trekd\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse.db', engine)
X = df['message']
Y = df.drop(columns=['id', 'message', 'original', 'genre'])

In [3]:
filepath = './data/DisasterResponse.db'
db = filepath.split('/')[-1]

In [4]:
# load data from database
engine = create_engine(f'sqlite:///{filepath}')
df = pd.read_sql_table(db, engine)
X = df['message']
Y = df.drop(columns=['id', 'message', 'original', 'genre'])

### 2. Write a tokenization function to process your text data

In [5]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english')) 
pattern = re.compile(r'[^a-zA-Z0-9]')

def tokenize(text):    
    text = pattern.sub(' ', text)
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(w, pos='v') for w in tokens if w not in stop_words]    
   # tokens = [stemmer.stem(w) for w in tokens]
    
    return tokens

In [6]:
# ind = 10
# print(tokenize(X.to_list()[ind]))
# print(X[ind])

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize, token_pattern=None, lowercase=True)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(ComplementNB())),
])

param = {
    'vect__ngram_range': [(1,1), (1,2), (2,2)],
    'vect__max_df':[0.6, 0.8, 1.0],
    'tfidf__norm': ['l1', 'l2', None],
    'tfidf__use_idf': [True, False],
    'clf__estimator__alpha': (0.2, 0.5, 0.8, 2.0), 
}

multi_accuracy = make_scorer(accuracy_score)
multi_precision = make_scorer(precision_score, average='macro', zero_division=0)
multi_recall = make_scorer(recall_score, average='macro', zero_division=0)
multi_fbeta = make_scorer(fbeta_score, beta=2, average='macro', zero_division=0)

# scoring = {'acc': multi_accuracy,
#           # 'prec': multi_precision,
#            'rec': multi_recall,
#            #'f1': 'f1',
#            'fbeta_2': multi_fbeta,
#             #'avg_prec':avg_precision,
#           }

In [8]:
cv = RandomizedSearchCV(pipeline,
                      param,                      
                      verbose=5,
                      error_score='raise',
                      return_train_score=False,
                      n_jobs=-1,
                      n_iter=3,
                      scoring=multi_fbeta,
                      random_state=41,
                      #refit='fbeta_2',
                     )

In [9]:
# cv = GridSearchCV(pipeline,
#                   param,                  
#                   verbose=5,
#                   error_score='raise',
#                   return_train_score=False,
#                   n_jobs=-1,
#                   scoring=scoring,
#                   refit='fbeta_2',
#                  # random_state=41
#                  )

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, random_state=41)

In [11]:
start = time.time()
cv.fit(X_train, y_train)
end = time.time()
total = (end-start)/60
print(f'Complete in {total} minutes.')

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Complete in 0.4102023681004842 minutes.


In [12]:
pd.DataFrame(cv.cv_results_).sort_values('rank_test_score').head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_vect__ngram_range,param_vect__max_df,param_tfidf__use_idf,param_tfidf__norm,param_clf__estimator__alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
2,4.309469,0.345492,0.918143,0.041714,"(2, 2)",0.6,True,None,0.2,"{'vect__ngram_range': (2, 2), 'vect__max_df': ...",0.308171,0.323531,0.319251,0.326200,0.319906,0.319412,0.006159,1
0,4.792975,0.058527,0.906774,0.035247,"(1, 1)",0.8,False,None,2.0,"{'vect__ngram_range': (1, 1), 'vect__max_df': ...",0.240079,0.233171,0.235115,0.245104,0.233680,0.237430,0.004549,2
1,5.009060,0.436304,1.037959,0.036226,"(1, 2)",0.8,False,l1,0.5,"{'vect__ngram_range': (1, 2), 'vect__max_df': ...",0.078978,0.078472,0.083044,0.081058,0.079214,0.080153,0.001689,3


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [40]:
pred = pd.DataFrame(cv.predict(X_test))

target_names = y_test.columns

for i, col in enumerate(y_test):
    print(f'-------{target_names[i]}-{i}-------\n')
    measure_dict = classification_report(y_test[col], pred[i], output_dict=True)

    try:
        precision = measure_dict['1']['precision']
    except:
        precision = 0.

    try:
        recall = measure_dict['1']['recall']
    except:
        recall = 0.

    try:
        f1_score = measure_dict['1']['f1-score']
    except:
        f1_score = 0.
        

    print(f'precision: {precision:0.2}    recall: {recall:0.2}    f1-score: {f1_score:0.2}')

-------related-0-------

precision: 0.86    recall: 0.84    f1-score: 0.85
-------request-1-------

precision: 0.46    recall: 0.72    f1-score: 0.56
-------offer-2-------

precision: 0.0075    recall: 0.15    f1-score: 0.014
-------aid_related-3-------

precision: 0.58    recall: 0.73    f1-score: 0.65
-------medical_help-4-------

precision: 0.24    recall: 0.42    f1-score: 0.31
-------medical_products-5-------

precision: 0.18    recall: 0.42    f1-score: 0.26
-------search_and_rescue-6-------

precision: 0.11    recall: 0.27    f1-score: 0.15
-------security-7-------

precision: 0.043    recall: 0.14    f1-score: 0.065
-------military-8-------

precision: 0.23    recall: 0.48    f1-score: 0.31
-------child_alone-9-------

precision: 0.0    recall: 0.0    f1-score: 0.0
-------water-10-------

precision: 0.25    recall: 0.65    f1-score: 0.37
-------food-11-------

precision: 0.34    recall: 0.69    f1-score: 0.45
-------shelter-12-------

precision: 0.28    recall: 0.61    f1-score

In [13]:
pred = pd.DataFrame(cv.predict(X_test))

target_names = y_test.columns

for i, col in enumerate(y_test):
    print(f'-------{target_names[i]}-{i}-------\n')
    print(classification_report(y_test[col], pred[i]))

-------related-0-------

              precision    recall  f1-score   support

           0       0.53      0.56      0.54      1553
           1       0.86      0.84      0.85      5001

    accuracy                           0.78      6554
   macro avg       0.69      0.70      0.70      6554
weighted avg       0.78      0.78      0.78      6554

-------request-1-------

              precision    recall  f1-score   support

           0       0.93      0.82      0.87      5407
           1       0.46      0.72      0.56      1147

    accuracy                           0.80      6554
   macro avg       0.70      0.77      0.72      6554
weighted avg       0.85      0.80      0.82      6554

-------offer-2-------

              precision    recall  f1-score   support

           0       1.00      0.92      0.96      6527
           1       0.01      0.15      0.01        27

    accuracy                           0.92      6554
   macro avg       0.50      0.53      0.48      6554
w

### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
# parameters = 

# cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [15]:
accuracy = multi_accuracy(cv, X_test, y_test)
precision = multi_precision(cv, X_test, y_test)
recall = multi_recall(cv, X_test, y_test)

print(f'macro-averaged accuracy: {accuracy:.2f} -- macro-averaged precision: {precision:.2f} -- macro-averaged recall: {recall:.2f}')

macro-averaged accuracy: 0.11 -- macro-averaged precision: 0.21 -- macro-averaged recall: 0.41


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [16]:
filename = r'./models/model.pkl'
pickle.dump(cv, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.